In [3]:
import cpp_soft_info

# Load Data

In [4]:
%reload_ext autoreload
%autoreload 2

In [5]:
from result_saver import SaverProvider
provider = SaverProvider()

In [15]:
import numpy as np
from Scratch import metadata_loader

DEVICE = 'ibm_torino'

md = metadata_loader(_extract=True, _drop_inutile=True)
# md = md[md["job_status"] == "JobStatus.DONE"]
# md = md[md["notebook_name"] == "bigger_rep_codes"]
max_distance = int(max(md.distance))
max_distance = 56
md = md[md["distance"] == max_distance]
md = md[md["backend_name"]==DEVICE]


md = md[md["shots"]==10]
md = md[:2]

md

,creation_date,notebook_name,backend_name,job_id,tags,shots,tags_xp,rep_delay,sampled_state,num_qubits,job_status,extra,optimization_level,code,distance,rounds,logical,layout,descr
2552,2023-12-03 10:49:34.289000+01:00,torino_repcode,ibm_torino,cnp4ybh9drj000843pyg,"[RC(56, 100), 10]",10.0,NaN,NaN,NaN,NaN,JobStatus.RUNNING,NaN,NaN,RepetitionCodeCircuit,56.0,100,1,"_is_hex=True, sherbrooke",diff rounds v2
2551,2023-12-03 10:49:14.052000+01:00,torino_repcode,ibm_torino,cnp4y6g8sy80008p4dd0,"[RC(56, 100), 10]",10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RepetitionCodeCircuit,56.0,100,0,"_is_hex=True, sherbrooke",diff rounds v2


In [16]:
memories = {}
jobs = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()
    jobs[mmr_name] = job_id

memory_str = "mmr_log_0"
memory = memories[memory_str][:]
job_id = jobs[memory_str]
print(memory.shape)

(10, 5556)


# Initialize the code

In [17]:
import stim
import pymatching

from soft_info import get_repcode_layout, get_repcode_IQ_map
from Scratch import create_or_load_kde_grid

d = max_distance
synd_rounds = max_distance
synd_rounds = 100

circuit = stim.Circuit.generated("repetition_code:memory",
                                 distance=d,
                                 rounds=synd_rounds,
                                 after_clifford_depolarization=0.1)

model = circuit.detector_error_model(decompose_errors=True)

# layout = get_repcode_layout(distance=max_distance, backend=provider.get_backend(DEVICE), _is_hex=True)

##### TORINO ####
distance = max_distance
import pickle
with open('longest_path_torino.pkl', 'rb') as f:
    path = pickle.load(f)
bounded_path = path[:2 * distance - 1]
layout = bounded_path[1::2] + bounded_path[::2] 


qubit_mapping = get_repcode_IQ_map(layout, synd_rounds) #Hardcoded for repetition codes
# print(qubit_mapping)
# print(len(qubit_mapping))

# Sligthly faster faster triple loop

In [18]:
other_date = None # if none then it will find the closest to the tobecalib_job date
# other_date = "2023-11-22T10:30:00" # "2023-11-22T" works too
grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, job_id, 5, 0.3, other_date=other_date)

# takes 10s

In [19]:
# get_counts
counts = cpp_soft_info.get_counts(memory, qubit_mapping, grid_dict, processed_scaler_dict, synd_rounds)

# Decode

In [20]:
matching = pymatching.Matching.from_detector_error_model(model)
# p_data = 6.869e-3 # mean sherbrooke noise
p_data = 1.15e-2
num_errors = cpp_soft_info.decode_IQ_shots(matching._matching_graph, memory, 
                                           synd_rounds, qubit_mapping, grid_dict, 
                                           processed_scaler_dict, p_data=p_data, p_mixed=p_data/1, #p_mixed=1e-80, for d=30
                                           common_measure=-1)
print("num_errors:", num_errors, "out of", len(memory), "shots")
 
# takes 5s

num_errors: 1 out of 10 shots


In [21]:
matching = pymatching.Matching.from_detector_error_model(model)
num_errors = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, memory, 
                                           synd_rounds, qubit_mapping, grid_dict, 
                                           processed_scaler_dict)
print("num_errors:", num_errors, "out of", len(memory), "shots")

# takes 1s

num_errors: 5 out of 10 shots


In [22]:
matching = pymatching.Matching.from_detector_error_model(model)

def weight_to_prob(weight):
    return 1/(1+np.exp(weight))

p_data = 6.869e-3 # mean sherbrooke ECR error
p_mixed = p_data/1 # Same as weighted
p_meas = 9.900e-3 # mean Readout assignment error

num_errors = cpp_soft_info.decode_IQ_shots_flat_informed(matching._matching_graph, memory, 
                                           synd_rounds, qubit_mapping, grid_dict, processed_scaler_dict,
                                           p_data, p_mixed, p_meas, common_measure=-1)
                                        
print("num_errors:", num_errors, "out of", len(memory), "shots")
         
# takes 1s

num_errors: 4 out of 10 shots
